In [1]:
import gc
import os

import keras
import mat73
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from nowcasting.unet import res1
from sklearn.model_selection import train_test_split
from tensorflow.keras import mixed_precision
from tqdm import tqdm


2022-10-29 18:03:08.203895: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-29 18:03:08.412348: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-29 18:03:09.316745: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-29 18:03:09.316832: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or direc

In [2]:
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
tf.config.optimizer.set_experimental_options({"layout_optimizer": False})

os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

policy = mixed_precision.Policy("mixed_float16")
mixed_precision.set_global_policy(policy)

print(tf.config.list_physical_devices('GPU'))

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 2080 Ti, compute capability 7.5
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
#%% load the data
mat = mat73.loadmat("../data/GD/1Deg_800Sample.mat")  # 8 time step estimation
X_1 = mat[
    "X_train"]  # (sample, time sequence, latitude, longitude, channel) here channels are 1: precipitation, 2: wind velocity in x direction, 3: wind velocity in y direction
y_1 = mat["y_train"]  # (sample, time sequence, lat, lon)

In [4]:
X_test = mat["X_test"]
y_test = mat["y_test"]
GFS = mat["GFS_test"]

X_train, X_val, y_train, y_val = train_test_split(X_1,
                                                  y_1,
                                                  test_size=0.15,
                                                  random_state=42)
print("Train feature", X_train.shape, "Train label", y_train.shape)
print("Validation feature", X_test.shape, "Validation label", y_val.shape)


Train feature (680, 12, 120, 120, 3) Train label (680, 8, 120, 120)
Validation feature (201, 12, 120, 120, 3) Validation label (120, 8, 120, 120)


In [5]:
del X_1, y_1, mat, X_val, y_val, GFS
gc.collect()

0

In [6]:
model = res1((12, 120, 120, 3), num_filters_base=8, dropout_rate=0.2)
model.summary()
# with open("modelsummary.txt", "w") as f:
#     model.summary(print_fn=lambda x: f.write(x + "\n"))

2022-10-29 18:03:53.373225: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-29 18:03:54.147873: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-10-29 18:03:54.147899: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:222] Using CUDA malloc Async allocator for GPU: 0
2022-10-29 18:03:54.148036: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9621 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:04:00.0, compute capability: 7.5


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 12, 120, 12  0           []                               
                                0, 3)]                                                            
                                                                                                  
 batch_normalization (BatchNorm  (None, 12, 120, 120  12         ['input_1[0][0]']                
 alization)                     , 3)                                                              
                                                                                                  
 conv_lstm1 (ConvLSTM2D)        (None, 12, 120, 120  3200        ['batch_normalization[0][0]']    
                                , 8)                                                          

In [7]:
#%%

model.compile(loss="mean_absolute_error",
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=["mae", "mse"])
#mean_absolute_error
checkpoint_filepath = "script_n1.h5"
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint(filepath=checkpoint_filepath,
                    verbose=1,
                    save_best_only=True,
                    save_weights_only=True)
]

#%%
results = model.fit(X_train,
                    y_train,
                    batch_size=8,
                    epochs=50,
                    callbacks=callbacks,
                    verbose=1,
                    validation_data=(X_test, y_test))


2022-10-29 18:04:00.982538: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1410048000 exceeds 10% of free system memory.
2022-10-29 18:04:02.829345: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1410048000 exceeds 10% of free system memory.


Epoch 1/50


2022-10-29 18:04:44.076464: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2022-10-29 18:04:44.878328: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


85/85 [==============================] - ETA: 0s - loss: 0.1066 - mae: 0.1066 - mse: 0.3699
Epoch 1: val_loss improved from inf to 0.08157, saving model to script_n1.h5
85/85 [==============================] - 151s 1s/step - loss: 0.1066 - mae: 0.1066 - mse: 0.3699 - val_loss: 0.0816 - val_mae: 0.0816 - val_mse: 0.2095 - lr: 0.0010
Epoch 2/50
85/85 [==============================] - ETA: 0s - loss: 0.1008 - mae: 0.1008 - mse: 0.3247
Epoch 2: val_loss improved from 0.08157 to 0.07833, saving model to script_n1.h5
85/85 [==============================] - 101s 1s/step - loss: 0.1008 - mae: 0.1008 - mse: 0.3247 - val_loss: 0.0783 - val_mae: 0.0783 - val_mse: 0.1946 - lr: 0.0010
Epoch 3/50
85/85 [==============================] - ETA: 0s - loss: 0.0991 - mae: 0.0991 - mse: 0.3157
Epoch 3: val_loss improved from 0.07833 to 0.07826, saving model to script_n1.h5
85/85 [==============================] - 101s 1s/step - loss: 0.0991 - mae: 0.0991 - mse: 0.3157 - val_loss: 0.0783 - val_mae: 0.0782